In [1]:
import leafmap.foliumap as leafmap

In [4]:
m = leafmap.Map(locate_control=True, latlon_control=True, draw_export=True, minimap_control=True)
m

In [5]:
start_date = '2022-08-15'
end_date = '2022-10-30'

In [10]:
import json

# Define the GeoJSON object.
geojson = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[67.527016,26.698999],[67.527016,27.105589],[68.092573,27.105589],[68.092573,26.698999],[67.527016,26.698999]]]}}]}

# Parse the GeoJSON object
parsed_json = json.loads(json.dumps(geojson))

# Extract the coordinates from the GeoJSON object
coordinates = parsed_json['features'][0]['geometry']['coordinates'][0]

# Create an ee.Geometry object using the coordinates
geometry = ee.Geometry.Polygon(coordinates)

# Print the geometry
print(geometry)

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              67.527016,
              26.698999
            ],
            [
              67.527016,
              27.105589
            ],
            [
              68.092573,
              27.105589
            ],
            [
              68.092573,
              26.698999
            ],
            [
              67.527016,
              26.698999
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})


In [11]:
image_collection = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2") \
        .filterDate(start_date, end_date) \
        .filterBounds(geometry)

In [12]:
median = image_collection.median()

In [13]:
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [14]:
dataset = apply_scale_factors(median)

In [15]:
dataset

In [16]:
imageRGB = dataset.visualize(**{'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min':0.0, 'max': 0.3})

In [19]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'image.tif')

In [21]:
import geemap
geemap.ee_export_image(imageRGB,filename=filename,scale= 50, region=geometry)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\oo_wa\Downloads\image.tif
